In [1]:
class link:
    def __init__(self, name, a0, a1):
        self.name = name
        self.travel_time = 0 # the travel time of the link
        self._flow = 0 # the flow on the link
        self._paths = [] # the paths using this link
        self._a0 = a0
        self._a1 = a1
        
    def add_path(self, path):
        self._paths.append(path)
    
    def update_flow(self):
        self._flow = 0
        for path in self._paths:
            self._flow += path.flow
    
    def update_travel_time(self):
        self.update_flow()
        self.travel_time = self._a0 + self._a1 * self._flow
        
    def __str__(self):
        return self.name

In [18]:
class path:
    def __init__(self, name, links):
        self.name = name
        self._travel_time = 0 # the travel time of the paths
        self.flow = 0 # the flow on the paths
        self._links = links # the paths using this link
        for link in links:
            link.add_path(self)
    
    def get_travel_time(self):
        return self._travel_time
    
    def update_travel_time(self):
        self._travel_time = 0
        for link in self._links:
            self._travel_time += link.travel_time
            
    def __str__(self):
        return self.name

In [19]:
class network:
    def __init__(self, paths, links):
        self.paths = paths
        self.links = links
    
    def update(self):
        for link in self.links:
            link.update_travel_time()
        for path in self.paths:
            path.update_travel_time()

In [20]:
ab = link('ab', 1, 1/100)
ac = link('ac', 2, 0)
bc = link('bc', 0.25, 0)
bd = link('bd', 2, 0)
cd = link('cd', 1, 1/100)

In [21]:
abcd = path('abcd', [ab, bc, cd])
abd = path('abd', [ab, bd])
acd = path('acd', [ac, cd])

In [22]:
braess = network([abd, acd, abcd], [ab, ac, bc, bd, cd])

In [23]:
braess.update()

In [24]:
#Nash equilibrium
abcd.flow = 50
abd.flow = 25
acd.flow = 25
braess.update()
print([path.get_travel_time() for path in braess.paths])

[3.75, 3.75, 3.75]


In [25]:
#social optimum
abcd.flow = 0
abd.flow = 50
acd.flow = 50
braess.update()
print([path.get_travel_time() for path in braess.paths])

[3.5, 3.5, 3.25]
